# Multi-point stress approximation - walkthrough of tutorial
This notebook focuses on dissembling the `mpsa` notebook. The equations and code is looked at in detail

# Equation and physics

The linear elasticity problem may be represented with the following equation,
$$\nabla \cdot \sigma = - \vec{f}, \quad \vec{x} \in \Omega$$
where $\vec{f}$ is a body force, and $\omega$ is the stress. Stress is in PorePy represented as a linear function of the displacement
$$\sigma = C : \vec{u}$$

*Porepy* uses the following conventions:
* *tension* is positive. I.e.: The Cartesian component of the traction $\vec{T} = \sigma \cdot \vec{n}$, for a direction $\vec{r}$ is a positive number if the inner product $\vec{T}\cdot\vec{r}$ is positive. (This is opposite to the convection in *Fundamentals of Rock Mechanics*)
* The displacement vector given by $\vec{u}$ points from the initial state $\vec{x}$ to the final state $\vec{x}^*$, i.e. $\vec{u} = \vec{x}^* - \vec{x}$.

The system is bounded by three possible boundary conditions; Neumann, Dirichlet, or Robin conditions. Dividing the boundary into disjoint sets $\Gamma_N$, $\Gamma_D$, and $\Gamma_R$ for the types of conditions, we have
\begin{equation}
\vec{u} = g_D, \quad \vec{x} \in \Gamma_D \\
\sigma \cdot n = g_N, \quad \vec{x} \in \Gamma_N \\
\sigma \cdot n + W \vec{u} = g_R, \quad \vec{x} \in \Gamma_R
\end{equation}

## Create a non-fractured grid

In [1]:
import numpy as np
import porepy as pp

# Create grid
n = 5
g = pp.CartGrid([n,n])
g.compute_geometry()

Next, we define the stress tensor $C$. The constutitive law,
\begin{equation}
\sigma = C : u = 2 \mu \epsilon + \lambda \text{trace} (\epsilon) I, \quad \epsilon = \frac{1}{2} (\nabla u + (\nabla u)^T)
\end{equation}
is implemented.

In [2]:
# Create stiffness matrix
lam = np.ones(g.num_cells)
mu = np.ones(g.num_cells)
C = pp.FourthOrderTensor(mu, lam)

Boundary conditions are set to
* Bottom boundary, dirichlet
* Otherwise, Neumann

In [ ]:
# Boundary type
dirich = np.ravel(np.argwhere(g.face_centers[1] < 1e-10))
bound = pp.BoundaryConditionVectorial(g, dirich)

**Question**: According to Wikipedia (https://en.wikipedia.org/wiki/Hooke%27s_law#Hooke's_law_for_continuous_media), the state of a medium around a point is represented by two second-order tensors, the *strain tensor* $\mathbf{\epsilon}$ (in lieu of the displacement $\mathbf{X}$) and the *stress tensor* $\mathbf{\sigma}$ (replacing the restoring force $\mathbf{F}$. The analogue of Hooke's spring law for continuous media is then
\begin{equation}
\mathbf{\sigma} = -\mathbf{c} \mathbf{\epsilon}
\end{equation}
where $\mathbf{c}$ is a *fourth-order tensor* (a linear map between second-order tensors) usually called the *stiffness tensor* or *elasticity tensor*

---
In linear elasticity (and thus PorePy), the basic assumption is that *components of stress are linear functions of the components of strain* (Jaeger, 2007). 